In [1]:
import numpy as np
from pychunkedgraph.graph import ChunkedGraph, ChunkedGraphMeta
from pychunkedgraph.graph.meta import ChunkedGraphMeta, GraphConfig, DataSource

ds = DataSource(
  EDGES= "gs://chunkedgraph/minnie65_0/edges",
  COMPONENTS= "gs://chunkedgraph/minnie65_0/components",
  WATERSHED= "gs://microns-seunglab/minnie65/ws_minnie65_0"
)
gc = GraphConfig(
  CHUNK_SIZE= [256, 256, 512]
)
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [2]:
cg.node_cache

{}

In [8]:
cg.get_parent(595601050719748126)

[<Cell value=668784544668258860 timestamp=2019-11-04 23:05:04.874000+00:00>, <Cell value=668784544666950194 timestamp=2019-11-04 02:38:55.729000+00:00>]


668784544668258860

In [15]:
cg.id_client.read_node(595601050719748096)

{_Attribute(key=b'counter', family_id='1', serializer=<pychunkedgraph.graph.utils.serializers.NumPyValue object at 0x7f76e5bf30d0>): [<Cell value=30 timestamp=2020-02-13 16:34:49.002000+00:00>,
  <Cell value=29 timestamp=2020-02-13 16:34:38.686000+00:00>,
  <Cell value=28 timestamp=2020-02-13 16:34:21.737000+00:00>,
  <Cell value=27 timestamp=2020-02-13 16:31:56.053000+00:00>,
  <Cell value=26 timestamp=2020-02-13 15:33:50.084000+00:00>,
  <Cell value=25 timestamp=2020-02-13 15:32:01.845000+00:00>,
  <Cell value=24 timestamp=2020-02-13 15:27:20.504000+00:00>,
  <Cell value=23 timestamp=2020-02-13 15:27:20.465000+00:00>,
  <Cell value=23 timestamp=2020-02-13 15:25:02.354000+00:00>,
  <Cell value=22 timestamp=2020-02-13 15:14:15.090000+00:00>,
  <Cell value=21 timestamp=2020-02-13 15:04:08.131000+00:00>,
  <Cell value=20 timestamp=2020-02-13 15:02:00.637000+00:00>,
  <Cell value=19 timestamp=2020-02-13 14:58:51.131000+00:00>,
  <Cell value=18 timestamp=2020-02-13 14:56:45.886000+00:00>,


In [14]:
cg.get_segment_id(np.uint64(595601050719748126))

30

In [7]:
cg.id_client.create_node_id(np.uint64(595601050719748096))


595601050719748126 595601050719748126


595601050719748126

In [11]:
cg.get_all_parents_dict(595601050719748126)

[<Cell value=668784544668258860 timestamp=2019-11-04 23:05:04.874000+00:00>, <Cell value=668784544666950194 timestamp=2019-11-04 02:38:55.729000+00:00>]
[<Cell value=864691139344265709 timestamp=2019-11-07 21:19:58.970000+00:00>]


{9: 668784544668258860, 12: 864691139344265709}

In [4]:
edges = cg.read_chunk_edges([cg.get_chunk_id(layer=2,x=50,y=100,z=0)])
edges = edges["between"].get_pairs()
nodes = np.unique(edges)
parents = cg.get_roots(nodes)
d = dict(zip(parents, nodes))
print(d)

{864691138338302132: 75582971913961693, 864691137629078617: 75512534450307707, 864691137995403800: 75582903194484819, 864691137995404568: 75582834475009044, 864691138741942725: 75582971913961542, 864691137995404824: 75582903194484963, 864691136334203586: 75582971913961503}


In [5]:
# edges = np.array([[94803535283198313, 94803466563722624]],dtype=np.uint64)
# between
edges = np.array([[94524946524577880, 94595315268752176]],dtype=np.uint64)

print(cg.get_cross_chunk_edges_layer(edges))
print(cg.get_roots(edges[0]))

[8]
[864691139327226349 864691139527061485]


In [6]:
# parents = np.uint64(edges[0][0])
# while parents and parents.size:
#     parents = cg.get_parent(parents)
#     print(parents)

In [7]:
from pychunkedgraph.graph.edits import add_edges
from pychunkedgraph.graph.utils.context_managers import TimeIt

edges = np.array(edges, dtype=np.uint64)
with TimeIt("add_edge"):
    print(add_edges(cg, atomic_edges=edges))

Start add_edge
   Start get_parents edges
   3.0279159545898438e-05

166582540562333752 166582540562333752

166652909306511416 166652909306511416

238589352450457653 238589352450457653

238730089938812979 238730089938812979
Cannot find root id 238589352450457653, 7, 2020-02-13 16:31:44.426000+00:00

525232306542084142 525232306542084142

310826448204070958 310826448204070958

382959934314119214 382959934314119214

455162629527175214 455162629527175214
Cannot find root id 455162629527175214, 7, 2020-02-13 16:31:49.256000+00:00

527484106355769390 527484106355769390
Cannot find root id 525232306542084142, 12, 2020-02-13 16:31:51.493000+00:00

864691128455135381 864691128455135381

600104650347118638 600104650347118638

668784544664518702 668784544664518702

595601050719748123 595601050719748123
Cannot find root id 668784544664518702, 12, 2020-02-13 16:31:58.354000+00:00

864691128455135382 864691128455135382
   [864691128455135381, 864691128455135382]
16.664423942565918


In [8]:
cg.get_parent(525232306542084136)

[<Cell value=595601050721427346 timestamp=2019-11-04 02:14:16.043000+00:00>, <Cell value=595601050720696680 timestamp=2019-11-04 00:15:57.627000+00:00>]


595601050721427346

In [9]:
cg.id_client.get_max_node_id(cg.get_chunk_id(525232306542084136))

525232306542084142

In [10]:
cg.get_children(525232306542084135)

array([164399529414623584], dtype=uint64)